In [13]:
# Install dependencies if needed
!pip install tensorflow scikit-learn joblib pandas numpy

# Import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:

train_data_path = "train_data.csv"
test_data_path = "test_data.csv"

df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)


features = [
    "CO2_Emissions", "Renewable_Energy", "Water_Consumption", "Waste_Management", "Biodiversity_Impact",
    "Gender_Diversity", "Employee_Satisfaction", "Community_Investment", "Safety_Incidents", "Labor_Rights",
    "Board_Diversity", "Executive_Pay_Ratio", "Transparency", "Shareholder_Rights", "Anti_Corruption",
    "Political_Donations"
]
target = "ESG_Score"


X_train = df_train[features].values
y_train = df_train[target].values

X_test = df_test[features].values
y_test = df_test[target].values


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


joblib.dump(scaler, "feature_scaler.pkl")

['feature_scaler.pkl']

In [ ]:
# Define Neural Network Model
model = keras.Sequential([
    layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(1)
])


model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])


print("\nTraining the model...\n")
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=100, batch_size=32, verbose=1)


model.save("esg_model.h5")
print("\n✅ Model trained and saved successfully!")


Training the model...

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 473.0423 - mean_absolute_error: 13.5012 - val_loss: 34.1589 - val_mean_absolute_error: 4.6555
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 31.0436 - mean_absolute_error: 4.4571 - val_loss: 26.1772 - val_mean_absolute_error: 4.0749
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 26.9871 - mean_absolute_error: 4.1431 - val_loss: 27.0199 - val_mean_absolute_error: 4.1441
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 26.9146 - mean_absolute_error: 4.1347 - val_loss: 25.8438 - val_mean_absolute_error: 4.0625
Epoch 5/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 26.9289 - mean_absolute_error: 4.1365 - val_loss: 25.0751 - val_mean_absolute_error: 3.9881
Epoch 6/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 26.0991 - mean_absolute_error: 4.0750 - val_loss: 25.7654 - val_mean_absolute_error: 4.0499
Epoch 7/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 26.4520 - mean_ab


✅ Model trained and saved successfully!


In [16]:
# Load trained model
model = keras.models.load_model("esg_model.h5")

# Evaluate on test data
loss, mae = model.evaluate(X_test, y_test)

# Calculate R² Score
from sklearn.metrics import r2_score
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f"\n📊 Model Evaluation:")
print(f"🔹 Mean Absolute Error (MAE): {mae:.2f}")
print(f"🔹 Mean Squared Error (MSE): {loss:.2f}")
print(f"🔹 R² Score (Accuracy): {r2:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 26.9461 - mean_absolute_error: 4.1239


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

📊 Model Evaluation:
🔹 Mean Absolute Error (MAE): 4.19
🔹 Mean Squared Error (MSE): 27.65
🔹 R² Score (Accuracy): 0.7204


In [ ]:
def predict_from_user_input():
    print("\n🔢 Enter the following values:")
    user_input = []

    for feature in features:
        value = float(input(f"Enter {feature}: "))
        user_input.append(value)

    # Convert to numpy array and reshape
    user_input = np.array(user_input).reshape(1, -1)

    # Load scaler and model
    scaler = joblib.load("feature_scaler.pkl")
    model = keras.models.load_model("esg_model.h5")

    # Scale the input
    user_scaled = scaler.transform(user_input)

    # Predict ESG Score
    predicted_score = model.predict(user_scaled)[0][0]

    # ✅ Denormalize the output
    predicted_score = (predicted_score * y_std) + y_mean

    print(f"\n🌍 Corrected Predicted ESG Score: {predicted_score:.2f}")

# Call the function
predict_from_user_input()



🔢 Enter the following values:
